In [1]:
from typing import Tuple, List, Union, Any, Optional, Dict, Literal, Callable, Type
import abc

from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.optim
import torch.utils.data
from torch import Tensor


from models.ridge_ALOOCV import fit_ridge_ALOOCV
from models.sandwiched_least_squares import sandwiched_LS_dense, sandwiched_LS_diag, sandwiched_LS_scalar

In [170]:
# Make regression data X, y
N = 1000
N_test = 1000
D = 10
d = 2
X = torch.randn(N, D)
X_test = torch.randn(N_test, D)
w_true = torch.randn(D, d)
y = (X @ w_true)**2 + torch.randn(N, d) * 0.1  # Adding some noise
y_test = (X_test @ w_true)**2 + torch.randn(N_test, d) * 0.1  # Adding some noise

In [171]:
from models.base import RidgeCVModule

#dense      
model = RidgeCVModule(
        lower_alpha=1e-6,
        upper_alpha=1e6,
        n_alphas=10,
     )

results = []
for i in range(10):
    model.fit(X, y)
    out = model(X)
    out_test = model(X_test)
    rmse = torch.sqrt(nn.functional.mse_loss(out, y))
    rmse_test = torch.sqrt(nn.functional.mse_loss(out_test, y_test))
    results.append(torch.tensor([rmse, rmse_test]))
results = torch.stack(results)
print("train rmse", results[:, 0].mean(), "std", results[:, 0].std())
print("test rmse", results[:, 1].mean(), "std", results[:, 1].std())
print("train", results[:, 0])
print("test", results[:, 1])

train rmse tensor(10.4807) std tensor(0.)
test rmse tensor(12.1221) std tensor(0.)
train tensor([10.4807, 10.4807, 10.4807, 10.4807, 10.4807, 10.4807, 10.4807, 10.4807,
        10.4807, 10.4807])
test tensor([12.1221, 12.1221, 12.1221, 12.1221, 12.1221, 12.1221, 12.1221, 12.1221,
        12.1221, 12.1221])


In [172]:
from models.base import RidgeModule

#dense      
model = RidgeModule(
        l2_reg = 1e+2,
     )

results = []
for i in range(10):
    model.fit(X, y)
    out = model(X)
    out_test = model(X_test)
    rmse = torch.sqrt(nn.functional.mse_loss(out, y))
    rmse_test = torch.sqrt(nn.functional.mse_loss(out_test, y_test))
    results.append(torch.tensor([rmse, rmse_test]))
results = torch.stack(results)
print("train rmse", results[:, 0].mean(), "std", results[:, 0].std())
print("test rmse", results[:, 1].mean(), "std", results[:, 1].std())
print("train", results[:, 0])
print("test", results[:, 1])

train rmse tensor(10.4798) std tensor(0.)
test rmse tensor(12.1217) std tensor(0.)
train tensor([10.4798, 10.4798, 10.4798, 10.4798, 10.4798, 10.4798, 10.4798, 10.4798,
        10.4798, 10.4798])
test tensor([12.1217, 12.1217, 12.1217, 12.1217, 12.1217, 12.1217, 12.1217, 12.1217,
        12.1217, 12.1217])


In [174]:
from models.random_feature_representation_boosting import GreedyRFRBoostRegressor

model = GreedyRFRBoostRegressor(
        in_dim=D,
        out_dim=d,
        hidden_dim=128,
        n_layers=3,
        randfeat_xt_dim=128,
        randfeat_x0_dim=128,
        l2_reg=0.1,
        l2_ghat=0.001,
        boost_lr=1.0,
        sandwich_solver="dense",
        feature_type="SWIM",
        upscale_type="iid",
    )        

results = []
for i in range(10):
    model.fit(X, y)
    out = model(X)
    out_test = model(X_test)
    rmse = torch.sqrt(nn.functional.mse_loss(out, y))
    rmse_test = torch.sqrt(nn.functional.mse_loss(out_test, y_test))
    results.append(torch.tensor([rmse, rmse_test]))
results = torch.stack(results)
print("train rmse", results[:, 0].mean(), "std", results[:, 0].std())
print("test rmse", results[:, 1].mean(), "std", results[:, 1].std())
print("train", results[:, 0])
print("test", results[:, 1])

train rmse tensor(1.4964) std tensor(0.1116)
test rmse tensor(2.8064) std tensor(0.2287)
train tensor([1.3970, 1.5329, 1.4186, 1.6478, 1.6805, 1.6075, 1.4511, 1.4148, 1.3857,
        1.4285])
test tensor([2.6418, 2.6560, 2.7093, 3.1223, 3.2234, 3.0175, 2.6113, 2.6443, 2.6364,
        2.8020])


In [6]:
# train rmse tensor(2.3312) std tensor(0.5462)
# test rmse tensor(3.3370) std tensor(0.6041)
# train tensor([2.5459, 1.9770, 2.1338, 3.6567, 1.9788, 2.5264, 2.1827, 1.9025, 2.6022,
#         1.8058])
# test tensor([3.7197, 2.8087, 2.9656, 4.6065, 2.9207, 3.3118, 3.1447, 3.0470, 4.0548,
#         2.7906])

In [175]:
from models.random_feature_representation_boosting import GreedyRFRBoostRegressor

model = GreedyRFRBoostRegressor(
        in_dim=D,
        out_dim=d,
        hidden_dim=128,
        n_layers=5,
        randfeat_xt_dim=128,
        randfeat_x0_dim=128,
        l2_reg=0.01,
        l2_ghat=0.001,
        boost_lr=1.0,
        sandwich_solver="diag",
        feature_type="SWIM",
        upscale_type="iid",
    )        

results = []
for i in range(10):
    model.fit(X, y)
    out = model(X)
    out_test = model(X_test)
    rmse = torch.sqrt(nn.functional.mse_loss(out, y))
    rmse_test = torch.sqrt(nn.functional.mse_loss(out_test, y_test))
    results.append(torch.tensor([rmse, rmse_test]))
results = torch.stack(results)
print("train rmse", results[:, 0].mean(), "std", results[:, 0].std())
print("test rmse", results[:, 1].mean(), "std", results[:, 1].std())
print("train", results[:, 0])
print("test", results[:, 1])

train rmse tensor(3.3916) std tensor(0.3645)
test rmse tensor(4.7551) std tensor(0.4072)
train tensor([3.0822, 3.7635, 3.3762, 3.0931, 3.4500, 4.1593, 3.4233, 2.8789, 3.4191,
        3.2703])
test tensor([4.3154, 5.1048, 4.8654, 4.5901, 4.6795, 5.6666, 4.7048, 4.2476, 4.8138,
        4.5627])


In [177]:
from models.random_feature_representation_boosting import GreedyRFRBoostRegressor

model = GreedyRFRBoostRegressor(
        in_dim=D,
        out_dim=d,
        hidden_dim=128,
        n_layers=5,
        randfeat_xt_dim=128,
        randfeat_x0_dim=128,
        l2_reg=0.01,
        l2_ghat=0.000001,
        boost_lr=1.0,
        sandwich_solver="scalar",
        feature_type="SWIM",
        upscale_type="iid",
    )        

results = []
for i in range(10):
    model.fit(X, y)
    out = model(X)
    out_test = model(X_test)
    rmse = torch.sqrt(nn.functional.mse_loss(out, y))
    rmse_test = torch.sqrt(nn.functional.mse_loss(out_test, y_test))
    results.append(torch.tensor([rmse, rmse_test]))
results = torch.stack(results)
print("train rmse", results[:, 0].mean(), "std", results[:, 0].std())
print("test rmse", results[:, 1].mean(), "std", results[:, 1].std())
print("train", results[:, 0])
print("test", results[:, 1])

train rmse tensor(8.9572) std tensor(0.0009)
test rmse tensor(10.8555) std tensor(0.0010)
train tensor([8.9585, 8.9572, 8.9562, 8.9567, 8.9579, 8.9557, 8.9582, 8.9563, 8.9577,
        8.9573])
test tensor([10.8568, 10.8556, 10.8542, 10.8560, 10.8560, 10.8541, 10.8563, 10.8541,
        10.8566, 10.8556])


In [181]:
from models.random_feature_representation_boosting import GradientRFRBoostRegressor

model = GradientRFRBoostRegressor(
        in_dim=D,
        out_dim=d,
        hidden_dim=128,
        n_layers=5,
        randfeat_xt_dim=128,
        randfeat_x0_dim=128,
        l2_reg=0.1,
        l2_ghat=0.000001,
        boost_lr=1.0,
        feature_type="SWIM",
        upscale_type="iid",
    )

results = []
for i in range(5):
    model.fit(X, y)
    out = model(X)
    out_test = model(X_test)
    rmse = torch.sqrt(nn.functional.mse_loss(out, y))
    rmse_test = torch.sqrt(nn.functional.mse_loss(out_test, y_test))
    results.append(torch.tensor([rmse, rmse_test]))
results = torch.stack(results)
print("train rmse", results[:, 0].mean(), "std", results[:, 0].std())
print("test rmse", results[:, 1].mean(), "std", results[:, 1].std())
print("train", results[:, 0])
print("test", results[:, 1])

train rmse tensor(0.7125) std tensor(0.0143)
test rmse tensor(1.8055) std tensor(0.1001)
train tensor([0.7191, 0.6952, 0.7272, 0.6992, 0.7219])
test tensor([1.8971, 1.6982, 1.7867, 1.7248, 1.9207])


# End2End

In [182]:
from old.models import End2EndMLPResNet

model = End2EndMLPResNet(
        in_dim=D,
        hidden_dim=128,
        bottleneck_dim=32,
        out_dim=d,
        n_blocks=3,
        loss = "mse",
        lr = 0.1,
        n_epochs = 30,
        end_lr_factor= 0.1,
        weight_decay = 0.001,
        batch_size = 64,
        )

results = []
for i in range(5):
    model.fit(X, y)
    out = model(X)
    out_test = model(X_test)
    rmse = torch.sqrt(nn.functional.mse_loss(out, y))
    rmse_test = torch.sqrt(nn.functional.mse_loss(out_test, y_test))
    results.append(torch.tensor([rmse, rmse_test]))
results = torch.stack(results)
print("train rmse", results[:, 0].mean(), "std", results[:, 0].std())
print("test rmse", results[:, 1].mean(), "std", results[:, 1].std())
print("train", results[:, 0])
print("test", results[:, 1])

100%|██████████| 30/30 [00:02<00:00, 14.76it/s]

train rmse tensor(1.5679) std tensor(1.1229)
test rmse tensor(3.0667) std tensor(0.7017)
train tensor([0.6448, 2.6953, 0.9447, 2.8827, 0.6717])
test tensor([2.4154, 4.1244, 2.9518, 3.3517, 2.4903])
